In [1]:
%run helper/setup_notebook.ipynb import display_table

Successfully connected to leetcode50 database.


In [2]:
display_table('Employee')

+-------------+---------------+--------------+
| employee_id | department_id | primary_flag |
+-------------+---------------+--------------+
|      1      |       1       |      N       |
|      2      |       1       |      Y       |
|      2      |       2       |      N       |
|      3      |       3       |      N       |
|      4      |       2       |      N       |
|      4      |       3       |      Y       |
|      4      |       4       |      N       |
+-------------+---------------+--------------+


#### *Employees can belong to multiple departments. When the employee joins other departments, they need to decide which department is their primary department. Note that when an employee belongs to only one department, their primary column is 'N'.*

### Write an SQL query to report all the employees with their primary department. For employees who belong to one department, report their only department.
```
+-------------+---------------+
| employee_id | department_id |
+-------------+---------------+
| 1           | 1             |
| 2           | 1             |
| 3           | 3             |
| 4           | 3             |
+-------------+---------------+
Explanation: 
- The Primary department for employee 1 is 1.
- The Primary department for employee 2 is 1.
- The Primary department for employee 3 is 3.
- The Primary department for employee 4 is 3.
```


In [3]:
%%sql 

SELECT 
    employee_id, 
    department_id 
FROM Employee 
WHERE employee_id NOT IN (
    SELECT employee_id 
    FROM Employee 
    WHERE primary_flag='Y'
    );

employee_id,department_id
1,1
3,3


In [4]:
%%sql 

SELECT 
    employee_id, 
    department_id
FROM Employee
WHERE primary_flag = 'Y'

employee_id,department_id
2,1
4,3


In [5]:
%%sql 

(SELECT 
    employee_id,
    department_id
FROM Employee
WHERE employee_id NOT IN (
    SELECT employee_id
    FROM Employee
    WHERE primary_flag = 'Y')
)

UNION 

(SELECT 
    employee_id,
    department_id
FROM Employee
WHERE primary_flag = 'Y')
ORDER BY employee_id;


employee_id,department_id
1,1
2,1
3,3
4,3


# Using Pandas

In [6]:
import pandas as pd 

In [7]:
employee_query = %sql SELECT * FROM Employee # type: ignore 
employee_df = employee_query.DataFrame()

display(employee_df)

,employee_id,department_id,primary_flag
0,1,1,N
1,2,1,Y
2,2,2,N
3,3,3,N
4,4,2,N
5,4,3,Y
6,4,4,N


In [8]:
employee_df.query("primary_flag == 'Y'")

,employee_id,department_id,primary_flag
1,2,1,Y
5,4,3,Y


In [9]:
multi_departement_df = employee_df.query("primary_flag == 'Y'")
multi_departement_df

,employee_id,department_id,primary_flag
1,2,1,Y
5,4,3,Y


### Filter the DataFrame using `isin()` and the Bitwise NOT `~` operators.

In [10]:
employee_df[employee_df['employee_id'].isin(multi_departement_df['employee_id'])]

,employee_id,department_id,primary_flag
1,2,1,Y
2,2,2,N
4,4,2,N
5,4,3,Y
6,4,4,N


In [11]:
employee_df[~employee_df['employee_id'].isin(multi_departement_df['employee_id'])]

,employee_id,department_id,primary_flag
0,1,1,N
3,3,3,N


In [12]:
single_department_df = employee_df[~employee_df['employee_id'].isin(multi_departement_df['employee_id'])]
single_department_df

,employee_id,department_id,primary_flag
0,1,1,N
3,3,3,N


### Union join using `concat()`: Allows us to concatenate DataFrames vertically (row-wise) to create a union of the two DataFrames. 

In [13]:
pd.concat([single_department_df, multi_departement_df])

,employee_id,department_id,primary_flag
0,1,1,N
3,3,3,N
1,2,1,Y
5,4,3,Y


In [14]:
# By setting ignore_index=True, the resulting DataFrame output has a new index generated for the concatenated rows.
output = pd.concat([single_department_df, multi_departement_df], ignore_index=True)
output

,employee_id,department_id,primary_flag
0,1,1,N
1,3,3,N
2,2,1,Y
3,4,3,Y


In [15]:
output.sort_values('employee_id')

,employee_id,department_id,primary_flag
0,1,1,N
2,2,1,Y
1,3,3,N
3,4,3,Y


In [16]:
output.sort_values('employee_id')[['employee_id', 'department_id']]

,employee_id,department_id
0,1,1
2,2,1
1,3,3
3,4,3


# NOTES

In [17]:
# Using mask() method 
employee_df.mask(employee_df['employee_id'].isin(multi_departement_df['employee_id']))

,employee_id,department_id,primary_flag
0,1.0,1.0,N
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,3.0,3.0,N
4,NaN,NaN,NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN


#### Notice that the `mask()` method converts the original department_id value from `int` to `float` because `NaN` values in pandas are represented as float values. In the end, `NaN` is a special ***floating-point*** value that represents missing or undefined data. The conversion is done to maintain the data type consistency.

In [18]:
masked_df = employee_df.mask(employee_df['employee_id'].isin(multi_departement_df['employee_id']))
masked_df.dropna()

,employee_id,department_id,primary_flag
0,1.0,1.0,N
3,3.0,3.0,N
